In [134]:
import numpy as np
from scipy.stats import multivariate_normal

# Cluster 1
c1_prior = 0.5
c1_bern_p = 0.3
c1_norm_mean = np.array([1, 1])
c1_norm_cov = np.array([[2, 0.5], [0.5, 2]])
c1_norm = multivariate_normal(c1_norm_mean, c1_norm_cov)

# Cluster 2
c2_prior = 0.5
c2_bern_p = 0.7
c2_norm_mean = np.array([0, 0])
c2_norm_cov = np.array([[1.5, 1], [1, 1.5]])
c2_norm = multivariate_normal(c2_norm_mean, c2_norm_cov)

# Observations
x1 = np.array([1, 0.6, 0.1])
x2 = np.array([0, -0.4, 0.8])
x3 = np.array([0, 0.2, 0.5])
x4 = np.array([1, 0.4, -0.1])

In [135]:
# E step

# x1
x1_posterior_c1 = c1_bern_p*c1_norm.pdf(x1[1:3])*c1_prior
x1_posterior_c2 = c2_bern_p*c2_norm.pdf(x1[1:3])*c2_prior
x1_total_prob = x1_posterior_c1 + x1_posterior_c2
x1_posterior_c1, x1_posterior_c2 = x1_posterior_c1/x1_total_prob, x1_posterior_c2/x1_total_prob

# x2
x2_posterior_c1 = (1-c1_bern_p)*c1_norm.pdf(x2[1:3])*c1_prior
x2_posterior_c2 = (1-c2_bern_p)*c2_norm.pdf(x2[1:3])*c2_prior
x2_total_prob = x2_posterior_c1 + x2_posterior_c2
x2_posterior_c1, x2_posterior_c2 = x2_posterior_c1/x2_total_prob, x2_posterior_c2/x2_total_prob

# x3
x3_posterior_c1 = (1-c1_bern_p)*c1_norm.pdf(x3[1:3])*c1_prior
x3_posterior_c2 = (1-c2_bern_p)*c2_norm.pdf(x3[1:3])*c2_prior
x3_total_prob = x3_posterior_c1 + x3_posterior_c2
x3_posterior_c1, x3_posterior_c2 = x3_posterior_c1/x3_total_prob, x3_posterior_c2/x3_total_prob

# x4
x4_posterior_c1 = c1_bern_p*c1_norm.pdf(x4[1:3])*c1_prior
x4_posterior_c2 = c2_bern_p*c2_norm.pdf(x4[1:3])*c2_prior
x4_total_prob = x4_posterior_c1 + x4_posterior_c2
x4_posterior_c1, x4_posterior_c2 = x4_posterior_c1/x4_total_prob, x4_posterior_c2/x4_total_prob

print(c1_bern_p*c1_norm.pdf(x1[1:3])*c1_prior)
print(c2_bern_p*c2_norm.pdf(x1[1:3])*c2_prior)
print(x1_posterior_c1, x1_posterior_c2)

0.009986294880455089
0.041866429997205
0.1925895872206616 0.8074104127793383


In [136]:
# M step

observations = np.array([x1[1:3], x2[1:3], x3[1:3], x4[1:3]]).T

total_posteriors_c1 = x1_posterior_c1 + x2_posterior_c1 + x3_posterior_c1 + x4_posterior_c1
total_posteriors_c2 = x1_posterior_c2 + x2_posterior_c2 + x3_posterior_c2 + x4_posterior_c2

weights_c1 = np.array([x1_posterior_c1, x2_posterior_c1, x3_posterior_c1, x4_posterior_c1])/total_posteriors_c1
weights_c2 = np.array([x1_posterior_c2, x2_posterior_c2, x3_posterior_c2, x4_posterior_c2])/total_posteriors_c2

new_c1_bern_p = sum(np.array([1, 0, 0, 1])*weights_c1)
new_c2_bern_p = sum(np.array([1, 0, 0, 1])*weights_c2)

new_c1_norm_mean = np.average(observations, axis=1, weights=weights_c1)
new_c2_norm_mean = np.average(observations, axis=1, weights=weights_c2)

new_c1_norm_cov = np.cov(observations, aweights=weights_c1, ddof=0)
new_c2_norm_cov = np.cov(observations, aweights=weights_c2, ddof=0)

new_c1_prior = total_posteriors_c1/(total_posteriors_c1+total_posteriors_c2)
new_c2_prior = total_posteriors_c2/(total_posteriors_c1+total_posteriors_c2)

print(new_c1_bern_p)
print(new_c1_norm_mean)
print(new_c1_norm_cov)
print(new_c1_prior)

0.2340394840854108
[0.026509   0.50712978]
[[ 0.14136501 -0.10540546]
 [-0.10540546  0.0960526 ]]
0.3861675546788702


In [137]:
# Ex 2

new_c1_norm = multivariate_normal(new_c1_norm_mean, new_c1_norm_cov)
new_c2_norm = multivariate_normal(new_c2_norm_mean, new_c2_norm_cov)

x_new = np.array([1, 0.3, 0.7])

x_new_posterior_c1 = new_c1_bern_p*new_c1_norm.pdf(x_new[1:3])*new_c1_prior
x_new_posterior_c2 = new_c2_bern_p*new_c2_norm.pdf(x_new[1:3])*new_c2_prior
total_x_new_prob = x_new_posterior_c1 + x_new_posterior_c2
x_new_posterior_c1, x_new_posterior_c2 = x_new_posterior_c1/total_x_new_prob, x_new_posterior_c2/total_x_new_prob

print("C1: ", x_new_posterior_c1)
print("C2: ", x_new_posterior_c2)

C1:  0.08028950846197537
C2:  0.9197104915380246


In [138]:
# Ex 3

# x1
new_x1_likelihood_c1 = new_c1_bern_p*new_c1_norm.pdf(x1[1:3])
new_x1_likelihood_c2 = new_c2_bern_p*new_c2_norm.pdf(x1[1:3])
new_x1_total_prob = new_x1_likelihood_c1 + new_x1_likelihood_c2
new_x1_likelihood_c1, new_x1_likelihood_c2 = new_x1_likelihood_c1/new_x1_total_prob, new_x1_likelihood_c2/new_x1_total_prob

# x2
new_x2_likelihood_c1 = (1 - new_c1_bern_p)*new_c1_norm.pdf(x2[1:3])
new_x2_likelihood_c2 = (1 - new_c2_bern_p)*new_c2_norm.pdf(x2[1:3])
new_x2_total_prob = new_x2_likelihood_c1 + new_x2_likelihood_c2
new_x2_likelihood_c1, new_x2_likelihood_c2 = new_x2_likelihood_c1/new_x2_total_prob, new_x2_likelihood_c2/new_x2_total_prob

# x3
new_x3_likelihood_c1 = (1 - new_c1_bern_p)*new_c1_norm.pdf(x3[1:3])
new_x3_likelihood_c2 = (1 - new_c2_bern_p)*new_c2_norm.pdf(x3[1:3])
new_x3_total_prob = new_x3_likelihood_c1 + new_x3_likelihood_c2
new_x3_likelihood_c1, new_x3_likelihood_c2 = new_x3_likelihood_c1/new_x3_total_prob, new_x3_likelihood_c2/new_x3_total_prob

# x4
new_x4_likelihood_c1 = new_c1_bern_p*new_c1_norm.pdf(x4[1:3])
new_x4_likelihood_c2 = new_c2_bern_p*new_c2_norm.pdf(x4[1:3])
new_x4_total_prob = new_x4_likelihood_c1 + new_x4_likelihood_c2
new_x4_likelihood_c1, new_x4_likelihood_c2 = new_x4_likelihood_c1/new_x4_total_prob, new_x4_likelihood_c2/new_x4_total_prob

print("X1: ", new_x1_likelihood_c1, new_x1_likelihood_c2)
print("X2: ", new_x2_likelihood_c1, new_x2_likelihood_c2)
print("X3: ", new_x3_likelihood_c1, new_x3_likelihood_c2)
print("X4: ", new_x4_likelihood_c1, new_x4_likelihood_c2)

X1:  0.13296862745986288 0.8670313725401372
X2:  0.8997775860290206 0.10022241397097947
X3:  0.6657785222758356 0.3342214777241645
X4:  0.01774039284027901 0.982259607159721
